<a href="https://colab.research.google.com/github/sehyun66/sehyun66/blob/master/steam_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 코랩을 사용할 때
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/MultiCampus/데이터 시각화3

Mounted at /content/drive
/content/drive/MyDrive/MultiCampus/데이터 시각화3


In [2]:
import pandas as pd

In [3]:
import pickle

In [5]:
df_steam_meta_con = pd.read_pickle('steam_meta.pkl')

In [28]:
df_steam_meta_con.head()

,appid,title,user,userscore
0,240,COUNTERSTRIKESOURCE,1,1
1,40,DEATHMATCHCLASSIC,1,3
2,10,COUNTERSTRIKE,1,2
3,130,HALFLIFEBLUESHIFT,1,3
4,40,DEATHMATCHCLASSIC,1,3


In [21]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 14.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633742 sha256=dd668fdf236be6e30866e8a3c68d07193e69aa13599a77fcef2ea1c6eae61ac9
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [22]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy

In [24]:
reader=Reader(rating_scale=(0,10))
data = Dataset.load_from_df(df_steam_meta_con[['appid','user','userscore']],
                           reader=reader)

In [29]:
cross_validate(SVD(),data,measures=['RMSE','MAE'],cv=5,verbose=1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.8615  2.8616  2.8608  2.8622  2.8599  2.8612  0.0008  
MAE (testset)     2.2116  2.2130  2.2110  2.2108  2.2130  2.2119  0.0010  
Fit time          100.84  111.13  103.71  103.63  103.08  104.48  3.49    
Test time         7.55    5.91    6.80    6.01    5.90    6.43    0.65    


{'fit_time': (100.84192299842834,
  111.13316226005554,
  103.70682454109192,
  103.63160467147827,
  103.07791781425476),
 'test_mae': array([2.21163609, 2.21295064, 2.21097534, 2.21079012, 2.21302248]),
 'test_rmse': array([2.8615055 , 2.86155234, 2.86082621, 2.8621815 , 2.85991656]),
 'test_time': (7.553109645843506,
  5.910796880722046,
  6.8043601512908936,
  6.007658958435059,
  5.896688461303711)}

In [6]:
from sklearn.decomposition import TruncatedSVD

In [7]:
from tqdm import tqdm

In [8]:
SVD2 = TruncatedSVD(n_components=20)

In [9]:
pdf_steam= pd.pivot_table(df_steam_meta_con,
                     index='appid',
                     columns='user',
                     values='userscore'
)

In [10]:
matrix=SVD2.fit_transform(pdf_steam.fillna(0))
matrix.shape

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'int', 'str']. An error will be raised in 1.2.
  FutureWarning,


(21630, 20)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
similarity_rate=cosine_similarity(matrix,matrix)

In [13]:
np.fill_diagonal(similarity_rate,0)
similarity_rate

array([[ 0.00000000e+00,  5.74591776e-01,  7.29265146e-01, ...,
        -8.15970054e-04,  6.12462797e-01,  1.93639035e-01],
       [ 5.74591776e-01,  0.00000000e+00,  9.39339623e-01, ...,
         1.98321923e-01,  4.33468565e-01,  5.26659845e-02],
       [ 7.29265146e-01,  9.39339623e-01,  0.00000000e+00, ...,
         2.51690141e-01,  5.49936049e-01,  7.83228168e-02],
       ...,
       [-8.15970054e-04,  1.98321923e-01,  2.51690141e-01, ...,
         0.00000000e+00,  1.96098033e-01, -1.49236907e-01],
       [ 6.12462797e-01,  4.33468565e-01,  5.49936049e-01, ...,
         1.96098033e-01,  0.00000000e+00,  3.96602350e-01],
       [ 1.93639035e-01,  5.26659845e-02,  7.83228168e-02, ...,
        -1.49236907e-01,  3.96602350e-01,  0.00000000e+00]])

In [14]:
pdf_steam2=pdf_steam.reset_index()
game_title=pdf_steam2['appid']

In [15]:
similarity_rate2=pd.DataFrame(data=similarity_rate,index=game_title,columns=game_title)
similarity_rate2

appid,10,20,30,40,50,60,70,130,220,240,...,2026370,2027100,2050470,2050680,2051440,2051970,2055270,2057930,2058650,2068930
appid,,,,,,,,,,,,,,,,,,,,,
10,0.000000,0.574592,0.729265,0.823365,0.335538,0.801024,0.831412,0.350146,0.768500,0.958411,...,0.167439,-0.028911,0.167439,-0.048001,-0.007412,0.074841,-0.115273,-0.000816,0.612463,0.193639
20,0.574592,0.000000,0.939340,0.641878,0.857645,0.641753,0.706057,0.852721,0.168515,0.574055,...,0.133308,0.257912,0.133308,0.041667,-0.014656,0.001900,-0.120805,0.198322,0.433469,0.052666
30,0.729265,0.939340,0.000000,0.751230,0.762335,0.742095,0.744005,0.759395,0.324003,0.733614,...,0.091316,0.142260,0.091316,-0.029579,-0.033457,0.031559,-0.111723,0.251690,0.549936,0.078323
40,0.823365,0.641878,0.751230,0.000000,0.588962,0.996512,0.874389,0.610427,0.707720,0.805287,...,0.068877,-0.014242,0.068877,-0.227773,-0.059871,0.075483,-0.099022,-0.051734,0.419605,0.106503
50,0.335538,0.857645,0.762335,0.588962,0.000000,0.613067,0.653177,0.997998,0.122746,0.314806,...,-0.046047,0.183542,-0.046047,-0.027055,-0.050823,-0.066828,0.010787,0.118128,0.074478,-0.000328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051970,0.074841,0.001900,0.031559,0.075483,-0.066828,0.083249,-0.029930,-0.054702,0.054061,0.082458,...,-0.226146,-0.198023,-0.226146,-0.313482,0.105559,0.000000,-0.134154,-0.060372,0.339442,0.430208
2055270,-0.115273,-0.120805,-0.111723,-0.099022,0.010787,-0.075151,-0.016487,0.017565,0.041723,-0.091488,...,0.251387,-0.059942,0.251387,0.705711,0.733530,-0.134154,0.000000,0.171529,-0.277706,-0.196565
2057930,-0.000816,0.198322,0.251690,-0.051734,0.118128,-0.040975,-0.159438,0.118295,-0.240559,-0.060750,...,0.112179,-0.322428,0.112179,0.190017,0.099967,-0.060372,0.171529,0.000000,0.196098,-0.149237


In [16]:
def recommend_game(appid, top_n):
   rec_game=similarity_rate2[appid].reset_index()
   rec_game = rec_game.rename(columns={"appid": "game",appid:'similarity'})
   rec_game=rec_game.sort_values(by='similarity',ascending=False).head(top_n)

   return rec_game

In [17]:
recommend_game(240,top_n=10)


,game,similarity
0,10,0.958411
16,440,0.896738
2961,316030,0.845483
1349,222608,0.838952
6,70,0.833539
3163,325958,0.829366
3417,337460,0.826189
69,3010,0.821276
14139,1094320,0.817350
15728,1225780,0.817350


In [18]:
# game1, game2, game3 추천 게임 - 유사도 테이블
def gentleman_ver2(game1, game2, game3):
  d1 = pd.DataFrame(recommend_game(game1, top_n=10), columns=['game', 'similarity']) # game1 추천목록 & 유사도
  d2 = pd.DataFrame(recommend_game(game2, top_n=10), columns=['game', 'similarity']) # game2 추천목록 & 유사도
  d3 = pd.DataFrame(recommend_game(game3, top_n=10), columns=['game', 'similarity']) # game3 추천목록 & 유사도
  df = pd.merge(d1, d2, on='game', how='outer') # d1, d2 merge
  df = pd.merge(df, d3, on='game', how='outer') # d2, d3 merge

  for game in df['game']: # ['game'] 행 하나씩 돌면서 
   if game in [game1, game2, game3]: # game이 game1, game2, game3에 해당하는 경우
      drop_index = df[df['game']==game].index
      df.drop(drop_index, axis='index', inplace=True) # 해당 행 drop

  # 추천 리스트 간 중복되는 게임이 없는 경우 각 추천리스트에서 2개씩 뽑음
  if len(df) == 30: 
    return [x for x in  recommend_game(game1, top_n=2)['game'].unique()]+ [x for x in recommend_game(game2, top_n=2)['game'].unique()] 
    + [x for x in recommend_game(game3, top_n=2)['game'].unique()]
  # 유사도 평균 내서 상위 6개 추출
  else : 
    df.fillna(0, inplace=True)
    df['mean'] = df.mean(axis=1, numeric_only=True)
    df.sort_values('mean', ascending=False, inplace=True)
    return list(df['game'][:6])

In [20]:
gentleman_ver2(240,3010,1225780)

[1787790, 1701720, 1446770, 1368780, 1315160, 1223830]